# 越南语对话模型训练

本示例展示如何在 NVIDIA GPU 服务器上配置 OpenXLab 下载越南语数据集与 LlamaFactory 为 Qwen1.5-0.5B-Chat 的 LoRA 模型训练准备。

## 项目概述

- **基础模型**: Qwen1.5-0.5B-Chat
- **微调方法**: LoRA (Low-Rank Adaptation)
- **训练数据**: 万卷丝绸数据集越南语部分
- **训练框架**: LlamaFactory
- **训练平台**: NVIDIA GPU (CUDA 12.6)

## 功能特性

- ✅ OpenXLab 数据集自动下载
- ✅ LlamaFactory 训练框架配置
- ✅ WebUI 可视化训练界面
- ✅ 多 GPU 训练支持

## 快速开始

1. 配置 Python 环境（PyTorch + CUDA）
2. 下载万卷丝绸越南语数据集
3. 安装 LlamaFactory
4. 启动 WebUI 进行训练配置

# 环境配置

## 版本要求

运行本训练脚本需要以下软件版本：

- **Python**: 3.11+
- **PyTorch**: 2.9.1+cu126
- **CUDA**: 12.6
- **transformers**: 4.57.1
- **peft**: 0.17.1
- **LlamaFactory**: 0.9.5.dev0

In [1]:
!pip list

Package                   Version      Editable project location
------------------------- ------------ ---------------------------------------------
accelerate                1.11.0
aiofiles                  24.1.0
aiohappyeyeballs          2.6.1
aiohttp                   3.13.3
aiosignal                 1.4.0
annotated-doc             0.0.4
annotated-types           0.7.0
antlr4-python3-runtime    4.9.3
anyio                     4.12.1
argon2-cffi               25.1.0
argon2-cffi-bindings      25.1.0
arrow                     1.4.0
asttokens                 3.0.1
async-lru                 2.0.5
attrs                     25.4.0
av                        16.0.1
babel                     2.17.0
beautifulsoup4            4.14.3
bleach                    6.3.0
brotli                    1.2.0
certifi                   2026.1.4
cffi                      2.0.0
charset-normalizer        3.4.4
click                     8.3.1
colorama                  0.4.6
comm                      0.2.3
conto


[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# 计算卡配置

使用 `nvidia-smi` 命令查看当前系统的 GPU 配置，包括 GPU 型号、驱动版本、显存使用情况和 GPU 利用率。

## 硬件配置

- **GPU 0**: NVIDIA GeForce RTX 3060 Ti (8GB)
- **GPU 1**: Tesla V100-SXM2-16GB (16GB)

推荐使用 Tesla V100 进行模型训练，显存更大，训练速度更快。

In [2]:
!nvidia-smi

Thu Jan  8 00:29:59 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.36                 Driver Version: 566.36         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060 Ti   WDDM  |   00000000:01:00.0  On |                  N/A |
|  0%   31C    P8              8W /  200W |     239MiB /   8192MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# 数据集下载

## OpenXLab

OpenXLab 是浦江实验室开源的数据集和模型托管平台，提供大规模中文多模态数据集下载服务。

## 万卷丝绸数据集 (WanJuanSiLu2O)

- **数据集名称**: OpenDataLab/WanJuanSiLu2O
- **越南语部分**: /raw/sft/vi
- **样本数量**: 约 23,000 条
- **数据格式**: JSONL

In [4]:
!pip install openxlab -i https://mirrors.tuna.tsinghua.edu.cn/pypi/web/simple

Looking in indexes: https://mirrors.tuna.tsinghua.edu.cn/pypi/web/simple
     -------------------------------------- 314.5/314.5 kB 1.4 MB/s eta 0:00:00
     -------------------------------------- 259.5/259.5 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 65.5/65.5 kB 1.8 MB/s eta 0:00:00
     -------------------------------------- 506.5/506.5 kB 3.2 MB/s eta 0:00:00
     ---------------------------------------- 62.8/62.8 kB ? eta 0:00:00
     -------------------------------------- 239.4/239.4 kB 7.2 MB/s eta 0:00:00
     -------------------------------------- 953.1/953.1 kB 6.7 MB/s eta 0:00:00
     ---------------------------------------- 77.1/77.1 kB 4.5 MB/s eta 0:00:00
     ---------------------------------------- 89.7/89.7 kB ? eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ------------

  DEPRECATION: crcmod is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  DEPRECATION: aliyun-python-sdk-core is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  DEPRECATION: oss2 is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can 

# 配置 OpenXLab 访问密钥

在使用 OpenXLab 下载数据集前，需要配置 AK（Access Key）和 SK（Secret Key）访问密钥。这些密钥用于验证用户身份和授权数据访问。

In [8]:
import os
os.environ["OPENXLAB_AK"] = "kz3xaykpe8dwomqpagen"
os.environ["OPENXLAB_SK"] = "pyjjazen5zvxowk2blz6ad19m4yomq71vdgm09d3"

In [9]:
"""
任务一：下载数据集
"""
import subprocess
import os
cmd = [
    "openxlab", "dataset", "download",
    "-r", "OpenDataLab/WanJuanSiLu2O",
    "-s", "/raw/sft/vi",
    "--target-path", "./TargetData/raw/"
]
print("开始下载数据...")
subprocess.run(cmd)
print("下载完成!")

开始下载数据...
下载完成!


# 训练框架安装

## LlamaFactory

LlamaFactory 是一个统一的大语言模型微调框架，支持多种训练方法（LoRA、全量微调、QLoRA等）、多种模型架构（LLaMA、Qwen、Baichuan等）以及 WebUI 可视化训练界面。

In [ ]:
!git clone --depth 1 https://github.com/hiyouga/LlamaFactory.git
!cd LlamaFactory
!pip install -e .
!pip install -r requirements/metrics.txt

# 启动 WebUI 训练界面

运行 `llamafactory-cli webui` 启动 LlamaFactory WebUI，在浏览器访问 http://localhost:7860 进行训练配置。

## 训练配置建议

| 参数 | 推荐值 |
|------|--------|
| 基础模型 | Qwen1.5-0.5B-Chat |
| 微调方法 | LoRA |
| LoRA Rank | 8-16 |
| LoRA Alpha | 16-32 |
| 学习率 | 5e-5 ~ 1e-4 |
| 训练轮数 | 3-5 epochs |
| 最大序列长度 | 512-1024 |

In [ ]:
!llamafactory-cli.exe webui